In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.utils import negative_sampling
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

Using backend: pytorch


In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../../data/dataset/',
                                 transform=T.ToSparseTensor())
dataset

PygNodePropPredDataset()

In [3]:
torch.cuda.is_available()

True

In [4]:
data = dataset[0]
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [5]:
split_idx = dataset.get_idx_split()
split_idx

{'train': tensor([     0,      1,      2,  ..., 169145, 169148, 169251]),
 'valid': tensor([   349,    357,    366,  ..., 169185, 169261, 169296]),
 'test': tensor([   346,    398,    451,  ..., 169340, 169341, 169342])}

In [6]:
data.adj_t = data.adj_t.to_symmetric()
data.adj_t

SparseTensor(row=tensor([     0,      0,      0,  ..., 169341, 169342, 169342]),
             col=tensor([   411,    640,   1162,  ..., 163274,  27824, 158981]),
             size=(169343, 169343), nnz=2315598, density=0.01%)

In [7]:
row, col, _ = data.adj_t.coo()

In [8]:
edge_index = torch.stack([col, row], dim=0)
edge_index

tensor([[   411,    640,   1162,  ..., 163274,  27824, 158981],
        [     0,      0,      0,  ..., 169341, 169342, 169342]])

In [9]:
edge_index.t()

tensor([[   411,      0],
        [   640,      0],
        [  1162,      0],
        ...,
        [163274, 169341],
        [ 27824, 169342],
        [158981, 169342]])

In [10]:
edge_index.shape

torch.Size([2, 2315598])

In [11]:
edge_index[:, 0]

tensor([411,   0])

In [12]:
edge_index.t()[0]

tensor([411,   0])

In [41]:
class SAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout):
        super(SAGE, self).__init__()

        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()

    def forward(self, x, adj_t):
        for conv in self.convs[:-1]:
            x = conv(x, adj_t)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, adj_t)
        return x


class LinkPredictor(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(LinkPredictor, self).__init__()
        self.lin = torch.nn.Linear(in_channels, out_channels)

    def reset_parameters(self):
        self.lin.weight.data.fill_(1)

    def forward(self, x_i, x_j):
        x = x_i * x_j
        x = self.lin(x)
        x = F.relu(x)
        return torch.sigmoid(x)

In [42]:
device=0
log_steps=1
num_layers=2
hidden_channels=50
dropout=0.5
batch_size=128 * 1024
lr=0.005
epochs=100
eval_steps=2
runs=10

In [43]:
device = f'cuda:{device}' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)
device

device(type='cuda', index=0)

In [44]:
adj_t = data.adj_t.to(device)

In [45]:
torch.manual_seed(12345)

In [46]:
model = SAGE(hidden_channels, hidden_channels,
             hidden_channels, num_layers,
             dropout).to(device)

In [47]:
emb = torch.nn.Embedding(data.num_nodes, hidden_channels).to(device)

In [48]:
predictor = LinkPredictor(hidden_channels, 1).to(device)

In [49]:
emb.weight.size(0), emb.weight.size(1)

(169343, 50)

In [50]:
torch.nn.init.xavier_uniform_(emb.weight)

Parameter containing:
tensor([[ 3.8116e-03, -1.7777e-04,  2.6052e-03,  ..., -9.5538e-04,
         -5.6698e-03,  2.9117e-03],
        [ 4.8286e-03,  5.7555e-03,  4.1475e-03,  ...,  2.2556e-03,
          2.7162e-03,  4.5704e-03],
        [ 1.6921e-03, -1.5087e-03,  3.1802e-03,  ..., -4.9685e-03,
         -5.1306e-04, -9.3337e-04],
        ...,
        [ 4.1685e-03,  3.3565e-03,  3.5653e-03,  ...,  3.1168e-03,
         -1.2510e-03, -3.7040e-03],
        [-6.9713e-04,  2.8990e-03,  4.6939e-03,  ...,  3.2265e-03,
          2.1993e-03,  4.2817e-03],
        [ 4.2867e-03, -3.3813e-03,  3.8860e-05,  ...,  2.8638e-03,
          5.3680e-04, -6.6582e-05]], device='cuda:0', requires_grad=True)

In [51]:
model.reset_parameters()
predictor.reset_parameters()

In [52]:
optimizer = torch.optim.Adam(
    list(model.parameters()) + list(emb.parameters()) +
    list(predictor.parameters()), lr=lr)

In [53]:
def train(model, predictor, x, adj_t, optimizer, batch_size):

    row, col, _ = adj_t.coo()
    edge_index = torch.stack([col, row], dim=0)

    model.train()
    predictor.train()

    pos_train_edge = edge_index.t().to(x.device)

    total_loss = total_examples = 0
    for perm in DataLoader(range(pos_train_edge.size(0)), batch_size,
                           shuffle=True):
        optimizer.zero_grad()

        h = model(x, adj_t)

        edge = pos_train_edge[perm].t()

        pos_out = predictor(h[edge[0]], h[edge[1]])
        pos_loss = -torch.log(pos_out + 1e-15).mean()

        edge = negative_sampling(edge_index, num_nodes=x.size(0),
                                 num_neg_samples=perm.size(0), method='sparse')

        neg_out = predictor(h[edge[0]], h[edge[1]])
        neg_loss = -torch.log(1 - neg_out + 1e-15).mean()

        loss = pos_loss + neg_loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(x, 1.0)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(predictor.parameters(), 1.0)

        optimizer.step()

        num_examples = pos_out.size(0)
        total_loss += loss.item() * num_examples
        total_examples += num_examples

    return total_loss / total_examples

In [54]:
for epoch in range(1, 1 + epochs):
    loss = train(model, predictor, emb.weight, adj_t,
                 optimizer, batch_size)
    
    if epoch % eval_steps == 0:
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}')

Epoch: 02, Loss: 1.1344
Epoch: 04, Loss: 1.0165
Epoch: 06, Loss: 0.9634
Epoch: 08, Loss: 0.9285
Epoch: 10, Loss: 0.9002
Epoch: 12, Loss: 0.8802
Epoch: 14, Loss: 0.8650
Epoch: 16, Loss: 0.8529
Epoch: 18, Loss: 0.8430
Epoch: 20, Loss: 0.8351
Epoch: 22, Loss: 0.8281
Epoch: 24, Loss: 0.8210
Epoch: 26, Loss: 0.8151
Epoch: 28, Loss: 0.8098
Epoch: 30, Loss: 0.8058
Epoch: 32, Loss: 0.8020
Epoch: 34, Loss: 0.7984
Epoch: 36, Loss: 0.7951
Epoch: 38, Loss: 0.7913
Epoch: 40, Loss: 0.7889
Epoch: 42, Loss: 0.7869
Epoch: 44, Loss: 0.7848
Epoch: 46, Loss: 0.7820
Epoch: 48, Loss: 0.7801
Epoch: 50, Loss: 0.7777
Epoch: 52, Loss: 0.7764
Epoch: 54, Loss: 0.7751
Epoch: 56, Loss: 0.7732
Epoch: 58, Loss: 0.7713
Epoch: 60, Loss: 0.7705
Epoch: 62, Loss: 0.7693
Epoch: 64, Loss: 0.7676
Epoch: 66, Loss: 0.7668
Epoch: 68, Loss: 0.7659
Epoch: 70, Loss: 0.7644
Epoch: 72, Loss: 0.7642
Epoch: 74, Loss: 0.7616
Epoch: 76, Loss: 0.7618
Epoch: 78, Loss: 0.7605
Epoch: 80, Loss: 0.7595
Epoch: 82, Loss: 0.7585
Epoch: 84, Loss:

In [55]:
predictor.lin.weight

Parameter containing:
tensor([[2.3051, 1.0035, 1.9316, 0.9331, 1.0529, 1.3620, 1.0817, 1.0687, 1.7008,
         1.2455, 1.2943, 0.9393, 0.9836, 1.1475, 2.3720, 1.0032, 0.9872, 1.0330,
         1.1058, 0.9621, 0.9796, 1.0139, 0.9832, 0.8654, 1.1625, 0.9785, 1.1312,
         0.9782, 0.9908, 1.0503, 0.8652, 1.0879, 1.1634, 0.9261, 0.9432, 0.9667,
         0.9000, 1.0575, 0.9860, 1.0430, 1.0898, 1.1150, 0.9332, 1.1843, 1.0730,
         0.9816, 0.9872, 0.9878, 1.3387, 0.9686]], device='cuda:0',
       requires_grad=True)

In [56]:
model.eval()

SAGE(
  (convs): ModuleList(
    (0): SAGEConv(50, 50)
    (1): SAGEConv(50, 50)
  )
)

In [57]:
emb.weight

Parameter containing:
tensor([[-0.1371, -0.2608, -0.2502,  ..., -0.0919, -0.2886,  0.1582],
        [ 0.3281,  0.0854, -0.0218,  ...,  0.0308, -0.0262, -0.1161],
        [-0.2098,  0.1723,  0.0692,  ...,  0.0392,  0.0853,  0.1854],
        ...,
        [-0.0155,  0.0959,  0.1951,  ...,  0.0955, -0.0562,  0.0080],
        [-0.0640,  0.0092,  0.0416,  ..., -0.0278,  0.0195,  0.1950],
        [-0.1959,  0.1542,  0.3159,  ...,  0.0519,  0.2112, -0.3311]],
       device='cuda:0', requires_grad=True)

In [58]:
h = model(emb.weight, adj_t)

In [59]:
h

tensor([[ 0.0323, -0.1698, -0.4031,  ..., -0.4262, -0.9858,  0.3831],
        [ 0.6392, -0.3035,  0.1097,  ..., -0.1709, -0.2094,  0.3562],
        [ 0.3483, -0.3747,  0.2352,  ..., -0.3968, -0.6598, -0.6173],
        ...,
        [-0.0883,  0.0505, -0.0584,  ...,  0.3078,  0.0079,  0.1374],
        [ 0.2262, -0.0766, -0.3615,  ..., -0.3752, -0.4662,  0.1222],
        [ 0.0800, -0.0258,  0.0686,  ..., -0.2138, -0.6644,  0.2232]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [60]:
predictor(h[0], h[411])

tensor([0.9922], device='cuda:0', grad_fn=<SigmoidBackward>)

In [61]:
predictor(h[0], h[2])

tensor([0.5000], device='cuda:0', grad_fn=<SigmoidBackward>)

In [62]:
(h[0] * h[2]).sum()

tensor(-0.3529, device='cuda:0', grad_fn=<SumBackward0>)

In [63]:
(h[0] * h[411]).sum()

tensor(7.5649, device='cuda:0', grad_fn=<SumBackward0>)

In [64]:
np.save('../../models/graphsage_link_pred/full_graphsage_linkpred_30_h.npy', h.cpu().detach().numpy())

In [65]:
torch.save(model, '../../models/graphsage_link_pred/30_model.pt')

In [66]:
torch.save(emb, '../../models/graphsage_link_pred/30_embedding.pt')

In [67]:
torch.save(predictor, '../../models/graphsage_link_pred/30_predictor.pt')